In [40]:
import pandas as pd
import numpy as np 
import os 
from pathlib import Path



In [5]:
# Data Section 1 - Team Overview 

# Find the teams for each Mens and Womens 
pd.set_option("display.max_columns",None)

# Team Info 
womens_teams = pd.read_csv("../data/WTeams.csv")
mens_teams = pd.read_csv("../data/MTeams.csv")

# Season Info 
womens_seasons = pd.read_csv("../data/Wseasons.csv")
mens_seasons = pd.read_csv("../data/MSeasons.csv")


# Seed Info 
mens_tourney_seeds = pd.read_csv("../data/MNCAATourneySeeds.csv")
womens_tourney_seeds = pd.read_csv("../data/WNCAATourneySeeds.csv")


# Regular Season Data - Each record: gamed
# Regular Seasons Games - All games before day num 132 
mens_reg_season_data = pd.read_csv("../data/MRegularSeasonCompactResults.csv")
womens_reg_season_data = pd.read_csv("../data/WRegularSeasonCompactResults.csv")

# Tourney Data 
mens_tourney_data = pd.read_csv("../data/MNCAATourneyCompactResults.csv")
womens_tourney_data = pd.read_csv("../data/WNCAATourneyCompactResults.csv")


# Regular Season Box Scores 
mens_reg_season_box_scores = pd.read_csv("../data/MRegularSeasonDetailedResults.csv")
womens_reg_season_box_scores = pd.read_csv("../data/WRegularSeasonDetailedResults.csv")

# Tournament Box Scores 
womens_tourney_box_scores = pd.read_csv("../data/WNCAATourneyDetailedResults.csv")
mens_tourney_box_scores = pd.read_csv("../data/MNCAATourneyDetailedResults.csv")

# Conferences 
conferences = pd.read_csv("../data/Conferences.csv")
mens_conferences = pd.read_csv("../data/MTeamConferences.csv")
womens_conferences = pd.read_csv("../data/WTeamConferences.csv")

# Alternate Spellings 
#mens_spellings = pd.read_csv("../data/MTeamSpellings.csv")
#womens_spellings = pd.read_csv("../data/WTeamSpellings.csv")

#Tourney Slots 
mens_tourney_slots = pd.read_csv("../data/MNCAATourneySlots.csv")
womens_tourney_slots = pd.read_csv("../data/WNCAATourneySlots.csv")
mens_tourney_seed_rounds = pd.read_csv("../data/MNCAATourneySeedRoundSlots.csv")

# Sample Submission
sample = pd.read_csv("../data/SampleSubmissionStage1.csv")

mens_reg_season_box_scores.dtypes




Season      int64
DayNum      int64
WTeamID     int64
WScore      int64
LTeamID     int64
LScore      int64
WLoc       object
NumOT       int64
WFGM        int64
WFGA        int64
WFGM3       int64
WFGA3       int64
WFTM        int64
WFTA        int64
WOR         int64
WDR         int64
WAst        int64
WTO         int64
WStl        int64
WBlk        int64
WPF         int64
LFGM        int64
LFGA        int64
LFGM3       int64
LFGA3       int64
LFTM        int64
LFTA        int64
LOR         int64
LDR         int64
LAst        int64
LTO         int64
LStl        int64
LBlk        int64
LPF         int64
dtype: object

In [ ]:


winning_teams = mens_reg_season_box_scores.groupby(['Season', 'WTeamID']).agg({
    'WScore': 'sum', 
    'LScore': 'sum',     
    'WFGM': 'sum', 
    'WFGA': 'sum',
    'WFGM3': 'sum',
    'WFGA3': 'sum',
    'WFTM': 'sum',
    'WFTA': 'sum',
    'WOR': 'sum', 
    'WDR': 'sum', 
    'WAst': 'sum', 
    'WTO': 'sum', 
    'LFGM': 'sum', 
    'LFGA': 'sum', 
    'LFGM3': 'sum',
    'LFGA3': 'sum',
    'LFTM': 'sum', 
    'LFTA': 'sum', 
    'LOR': 'sum', 
    'LDR': 'sum', 
    'LTO': 'sum'
}).reset_index() 

winning_teams['Wins'] = mens_reg_season_box_scores.groupby(['Season', 'WTeamID']).size().values

losing_teams = mens_reg_season_box_scores.groupby(['Season', 'LTeamID']).agg({  
    'LScore': 'sum',  
    'WScore': 'sum', 
    'LFGM': 'sum', 
    'LFGA': 'sum', 
    'LFGM3': 'sum',
    'LFGA3': 'sum',
    'LFTM': 'sum', 
    'LFTA': 'sum', 
    'LOR': 'sum', 
    'LDR': 'sum', 
    'LTO': 'sum',
    'LAst': 'sum',  
    'WFGM': 'sum', 
    'WFGA': 'sum',
    'WFGM3': 'sum',
    'WFGA3': 'sum',
    'WFTM': 'sum',
    'WFTA': 'sum',
    'WOR': 'sum', 
    'WDR': 'sum', 
    'WTO': 'sum'
}).reset_index()

losing_teams['Losses'] = mens_reg_season_box_scores.groupby(['Season', 'LTeamID']).size().values

merged_df = pd.merge(winning_teams, losing_teams, left_on=['Season', 'WTeamID'], right_on = ['Season', 'LTeamID'], how='outer', suffixes=('_1', '_2'))

# Combine the Stats for Winning and Losing games 
# Combine the Stats for Winning and Losing games 
merged_df['Points_Scored'] = merged_df['WScore_1'] + merged_df['LScore_2']
merged_df['FG_Made'] = merged_df['WFGM_1'] + merged_df['LFGM_2']
merged_df['FG_Atts'] = merged_df['WFGA_1'] + merged_df['LFGA_2']
merged_df['Threes_Made'] = merged_df['WFGM3_1'] + merged_df['LFGM3_2']
merged_df['Threes_Attempted'] = merged_df['WFGA3_1'] + merged_df['LFGA3_2']
merged_df['FreeThrows_Made'] = merged_df['WFTM_1'] + merged_df['LFTM_2']
merged_df['FreeThrows_Attempted'] = merged_df['WFTA_1'] + merged_df['LFTA_2']
merged_df['Offensive_Rebounds'] = merged_df['WOR_1'] + merged_df['LOR_2']
merged_df['Defensive_Rebounds'] = merged_df['WDR_1'] + merged_df['LDR_2']
merged_df['Assists'] = merged_df['WAst'] + merged_df['LAst']
merged_df['Turnovers'] = merged_df['WTO_1'] + merged_df['LTO_2']
merged_df['Points_Allowed'] = merged_df['LScore_1'] + merged_df['WScore_2']
merged_df['Opposing_FG_Made'] = merged_df['LFGM_1'] + merged_df['WFGM_2']
merged_df['Opposing_FG_Atts'] = merged_df['LFGA_1'] + merged_df['WFGA_2']
merged_df['Opposing_Threes_Made'] = merged_df['LFGM3_1'] + merged_df['WFGM3_2']
merged_df['Opposing_Threes_Attempted'] = merged_df['LFGA3_1'] + merged_df['WFGA3_2']
merged_df['Opposing_FreeThrows_Made'] = merged_df['LFTM_1'] + merged_df['WFTM_2']
merged_df['Opposing_FreeThrows_Attempted'] = merged_df['LFTA_1'] + merged_df['WFTA_2']
merged_df['Opposing_Offensive_Rebounds'] = merged_df['LOR_1'] + merged_df['WOR_2']
merged_df['Opposing_Defensive_Rebounds'] = merged_df['LDR_1'] + merged_df['WDR_2']
merged_df['Opposing_Turnovers'] = merged_df['LTO_1'] + merged_df['WTO_2']

merged_df['TeamID'] = merged_df['WTeamID'].combine_first(merged_df['LTeamID'])

output = [
    'Season', 'Losses', 'Wins', 'TeamID', 'Points_Scored', 'FG_Made', 'FG_Atts', 
    'Threes_Made', 'Threes_Attempted', 'FreeThrows_Made', 
    'FreeThrows_Attempted', 'Offensive_Rebounds', 'Defensive_Rebounds', 
    'Assists', 'Turnovers', 'Points_Allowed', 'Opposing_FG_Made', 
    'Opposing_FG_Atts', 'Opposing_Threes_Made', 'Opposing_Threes_Attempted', 
    'Opposing_FreeThrows_Made', 'Opposing_FreeThrows_Attempted', 
    'Opposing_Offensive_Rebounds', 'Opposing_Defensive_Rebounds', 'Opposing_Turnovers'
]

team_summary_stats = merged_df.copy() 
team_summary_stats = team_summary_stats[output]

team_summary_stats['Total_Games'] = team_summary_stats['Wins'] + team_summary_stats['Losses']
team_summary_stats['Points_Per_Game'] = team_summary_stats['Points_Scored'] / team_summary_stats['Total_Games']
team_summary_stats['Win_Percentage'] = team_summary_stats['Wins'] / team_summary_stats['Total_Games']
team_summary_stats['FG_Percentage'] = (team_summary_stats['FG_Made'] / team_summary_stats['FG_Atts']) * 100
team_summary_stats['Threes_Per_Game'] = team_summary_stats['Threes_Attempted'] / team_summary_stats['Total_Games']
team_summary_stats['Turnovers_Per_Game'] = team_summary_stats['Turnovers'] / team_summary_stats['Total_Games']
team_summary_stats['Three_Point_Percentage'] = (team_summary_stats['Threes_Made'] / team_summary_stats['Threes_Attempted']) * 100
team_summary_stats['Free_Throws_Per_Game'] = team_summary_stats['FreeThrows_Attempted'] / team_summary_stats['Total_Games']
team_summary_stats['Free_Throw_Percentage'] = (team_summary_stats['FreeThrows_Made'] / team_summary_stats['FreeThrows_Attempted']) * 100
team_summary_stats['Offensive_Rebound_Rate'] = (team_summary_stats['Offensive_Rebounds'] / (team_summary_stats['Offensive_Rebounds'] + team_summary_stats['Opposing_Defensive_Rebounds'])) * 100
team_summary_stats['Defensive_Rebound_Rate'] = (team_summary_stats['Defensive_Rebounds'] / (team_summary_stats['Defensive_Rebounds'] + team_summary_stats['Opposing_Offensive_Rebounds'])) * 100
team_summary_stats['Opp_FG_Percentage'] = (team_summary_stats['Opposing_FG_Made'] / team_summary_stats['Opposing_FG_Atts']) * 100
team_summary_stats['Opp_Three_Point_Percentage'] = (team_summary_stats['Opposing_Threes_Made'] / team_summary_stats['Opposing_Threes_Attempted']) * 100
team_summary_stats['Opp_Free_Throws_Per_Game'] = team_summary_stats['Opposing_FreeThrows_Attempted'] / team_summary_stats['Total_Games']
team_summary_stats['Opp_Turnovers_Per_Game'] = team_summary_stats['Opposing_Turnovers'] / team_summary_stats['Total_Games']
team_summary_stats['Effective_FG_Percentage'] = ((team_summary_stats['FG_Made'] + 0.5 * team_summary_stats['Threes_Made']) / team_summary_stats['FG_Atts']) * 100

team_summary_stats.fillna(0, inplace=True)

# Add team names to records so external data can be joined in 
mens_teams_subset = mens_teams[['TeamID', 'TeamName']]
team_summary_stats = team_summary_stats.merge(mens_teams_subset, on=['TeamID'], how='left')

percentage_columns = [
    'Season', 'TeamName', 'TeamID', 'Win_Percentage', 'Points_Per_Game', 'FG_Percentage', 'Threes_Per_Game', 'Three_Point_Percentage', 
    'Free_Throws_Per_Game', 'Free_Throw_Percentage', 'Offensive_Rebound_Rate', 'Defensive_Rebound_Rate', 'Effective_FG_Percentage',
    'Turnovers_Per_Game','Opp_FG_Percentage', 'Opp_Three_Point_Percentage', 'Opp_Free_Throws_Per_Game', 'Opp_Turnovers_Per_Game'
]

team_summary_stats = team_summary_stats[percentage_columns]

# Add Conferences and each record and one-hot encode them 
team_summary_stats = team_summary_stats.merge(mens_conferences, on=['Season', 'TeamID'], how='left')
team_summary_stats = pd.get_dummies(team_summary_stats, columns=['ConfAbbrev'], prefix='Conf', dtype=int)

team_summary_stats[team_summary_stats['TeamID'] == 1102][team_summary_stats['Season'] == 2003]

output_dir = Path("..") / "data" / "preprocessing"
output_dir.mkdir(parents=True, exist_ok =True)
output_path = output_dir / "mens_summary_season_data.csv"
team_summary_stats.to_csv(output_path, index=False)



/var/folders/mm/p9j4d6_x5b93fg7gz7zjjdkm0000gn/T/ipykernel_87661/1633084256.py:129: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  team_summary_stats[team_summary_stats['TeamID'] == 1102][team_summary_stats['Season'] == 2003]
